In [2]:
# Load in our libraries
import pandas as pd
import numpy as np
from datetime import datetime
import math
# import re
# import random as random
# import xgboost as xgb

# import warnings
# warnings.filterwarnings('ignore')

# # Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold
# from sklearn.metrics import f1_score

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 評価関数
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# 2. 築年月の数値を整数に変換し、NaNの処理を加えたdatetime形式変換関数を定義
def yyyymm_to_datetime(yyyymm_num):
    if math.isnan(yyyymm_num):  # NaNをチェック
        return None  # NaNの場合はNoneを返す
    yyyymm_int = int(yyyymm_num)  # 小数点以下を削除して整数に変換
    yyyymm_str = str(yyyymm_int)  # 整数を文字列に変換
    return datetime.strptime(yyyymm_str, "%Y%m")  # 文字列をdatetimeに変換

In [4]:
# 入力
train = pd.read_csv("../data/raw/train.csv")
test  = pd.read_csv("./data/raw/test.csv")
full_data = [train, test]

C:\Users\y.kasashima\AppData\Local\Temp\ipykernel_17988\1264471643.py:2: DtypeWarning: Columns (39,40,42,55,58) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/raw/train.csv")


FileNotFoundError: [Errno 2] No such file or directory: './data/raw/test.csv'

In [20]:
# 2. snapshot_window_angleをワンホットエンコーディングに変換
df_encoded = pd.get_dummies(train, columns=['building_structure'], prefix='bs')

# 3. 結果を確認
print(df_encoded)


        target_ym  money_room building_id  building_status  \
0          202007       38000     a000002                1   
1          202107      103000     a000004                1   
2          202101       50000     a000005                1   
3          202007       37000     a000008                1   
4          202101       33000     a000008                1   
...           ...         ...         ...              ...   
584502     202001       64000     a545390                1   
584503     202107       59000     a545392                1   
584504     202001       94000     a545393                1   
584505     201907       42000     a545394                1   
584506     202101       42000     a545394                1   

       building_create_date building_modify_date  building_type building_name  \
0       2014-06-27 21:18:41  2024-05-25 00:45:07              3   Plaisirりんくう   
1       2017-02-05 20:43:26  2024-08-17 12:12:51              1      向原パークハイツ   
2       2014

In [5]:
# 追加済み変数
# ・unit_area : 専有面積
# ・year_built_num : 築年月数(築年月から経過月数を算出)
# ・room_count : 間取部屋数
# ・madori_number_all : 間取部屋数(代表)
# ・flg_new : 新築・未入居フラグ
# ・walk_distance1 : 徒歩距離1
cols = ['unit_area', 'year_built_num', 'room_count', 'madori_number_all', 'flg_new', 'walk_distance1']

In [15]:
# 前処理
# 築経過月数の算出
current_date = datetime.now()
train['year_built_dt'] = train['year_built'].apply(yyyymm_to_datetime)
train['year_built_num'] = train['year_built_dt'].apply(lambda x: (current_date.year - x.year) * 12 + current_date.month - x.month)



In [17]:
train["dwelling_unit_window_angle"].head()

0    5.0
1    5.0
2    3.0
3    6.0
4    6.0
Name: dwelling_unit_window_angle, dtype: float64